In [237]:
import os
import numpy as np
import pandas as pd
from mycolorpy import colorlist as mcp
import matplotlib.pyplot as plt

from mylibs import tools, functions as f
from mylibs.tools import Experiment

pd.set_option('mode.chained_assignment', None)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('font', size=10)
plt.rc('axes', titlesize=16)
plt.rc('axes', labelsize=15)
plt.rc('legend', fontsize=10)
plt.rcParams["figure.figsize"] = (10, 7)

In [245]:
from scipy.interpolate import LinearNDInterpolator, CloughTocher2DInterpolator,griddata
from matplotlib.colors import LightSource
from matplotlib import cbook, cm

In [ ]:
%matplotlib qt

# Collect

In [ ]:
def to_df(experiment:Experiment):
    df= experiment.d
    k = 1
    df['Viscosity'] = k * (df['x'] + 273.15) / (df['y'] * 0.001)
    df['compound'] = experiment.info['compound']
    df['rho'] = experiment.info['rho']
    df['w'] = experiment.info['w']
    df['D0'] = experiment.info['D0']
    df['E'] = experiment.info['E']
    df.rename(columns={'x': 'Temperature', 'y': 'D'}, inplace=True)
    return df
    


In [ ]:
# OCM
ocm_list = []
for i in range(9):
    path = f'D:\WORKS\Diplom-work\Experiments\OCM_viscosity\OCM{i}'
    file = [f for f in os.listdir(path) if '.hdf5' in f][0]

    exp = Experiment()
    exp.load_hdf5(f'{path}\{file}')
    ocm_list.append(to_df(exp))

ocm = pd.concat(ocm_list)

In [ ]:
# PETA
peta_list = []
for i in range(5):
    path = f'D:\WORKS\Diplom-work\Experiments\PETA,TMAG_viscosity\PETA{i}'
    file = [f for f in os.listdir(path) if '.hdf5' in f][0]

    exp = Experiment()
    exp.load_hdf5(f'{path}\{file}')
    peta_list.append(to_df(exp))
    
peta = pd.concat(peta_list)

In [ ]:
# Butanol
but= Experiment()
but.load_hdf5('D:\Works\Diplom-work\Experiments\BUTANOL\data_BUT_up.hdf5')
but= to_df(but)


In [ ]:
all_df = pd.concat([ocm, peta,but]).reset_index(drop=True)[[
    'compound', 'w', 'Temperature', 'time', 'D', 'Viscosity', 'rho', 'D0', 'E', 'Viscosity_verbose'
]]


In [ ]:
df = (all_df[['compound','w','Temperature','D','Viscosity','rho','D0','E']]
    .groupby(by=['compound','Temperature','D','w'])
    .mean()
    .reset_index()
)

# 2D

In [ ]:
# 2D surface
compound_filter = (df['compound'] == 'ocm')
# |(df['compound'] == 'butanol')
data = df[compound_filter]

xy= data[['Temperature','w']]
z= data['D']
X = np.linspace(min(xy['Temperature']),max(xy['Temperature']))
Y = np.linspace(min(xy['w']),max(xy['w']))
X ,Y= np.meshgrid(X, Y)
f = LinearNDInterpolator(xy,z)
Z = f(X,Y)
plt.pcolormesh(X, Y, Z)

In [253]:
# Diffusion plots
name = 'OCM-2'
# +' + Butanol-1'

compound_filter = (
    (df['compound'] == 'peta')
    |(df['compound'] == 'butanol')
) #yapf: disable

# Plot
data = df[compound_filter]
D_mean = df['D'].mean()
grouped = data.groupby(by=['w'])
colors_list = mcp.gen_color(cmap="turbo", n=len(grouped))

fig, ax = plt.subplots()
for _, group in grouped:
    # print()
    w = group['w'].mean()
    T = group['Temperature']
    color = colors_list.pop()

    D = group['D'] / D_mean
    ax.scatter(
        T,
        D,
        c=color,
        s=2,
    )

    E = group['E'].mean()
    D0 = group['D0'].mean() / D_mean
    D_ols = D0 * np.exp(-E / (8.314 * (T+273.15)))
    ax.plot(
        T,
        D_ols,
        label=f"{w: 5.2f}",
        c=color,
    )

    ax.set_title(f"{name}: diffusion plots")
    ax.set_xlabel('Temperature [C]')
    ax.set_ylabel('Diffusion')
plt.legend()

# 3D T,w=D,rho

In [ ]:
import plotly
import plotly.graph_objects as go

fig = go.Figure()
fig.update_layout(hovermode="x unified", margin=dict(l=0, r=0, b=0, t=0, pad=0))


# OCM
compound_filter = (df['compound'] == 'ocm')
# |(df['compound'] == 'butanol')
data = df[compound_filter]

data['D'] = data['D'] / data['D'].mean()

xy = data[['Temperature', 'w']]
x = data['Temperature']
y = data['w']
x1 = np.linspace(x.min(), x.max(), 51)
y1 = np.linspace(y.min(), y.max(), 51)
X, Y = np.meshgrid(x1, y1)

z = data['D']
fz = CloughTocher2DInterpolator(xy, z)
Z = fz(X, Y)

r = data['rho']
fr = CloughTocher2DInterpolator(xy, r,)
#  fill_value=1
R = fr(X, Y)


fig.add_trace(go.Surface(
    x=x1,
    y=y1,
    z=Z,
    colorscale ='Reds'
    # surfacecolor=R,
))

# PETA
compound_filter = (df['compound'] == 'peta')
# |(df['compound'] == 'butanol')
data = df[compound_filter]

data['D'] = data['D'] / data['D'].mean()

xy = data[['Temperature', 'w']]
x = data['Temperature']
y = data['w']
x1 = np.linspace(x.min(), x.max(), 51)
y1 = np.linspace(y.min(), y.max(), 51)
X, Y = np.meshgrid(x1, y1)

z = data['D']
fz = CloughTocher2DInterpolator(xy, z)
Z = fz(X, Y)

r = data['rho']
fr = CloughTocher2DInterpolator(xy, r,)
#  fill_value=1
R = fr(X, Y)


fig.add_trace(go.Surface(
    x=x1,
    y=y1,
    z=Z,
    colorscale ='Blues'
    # surfacecolor=R,
))

fig.show()

# Rework massively

In [ ]:
import os
for i in range(5):

    exp = tools.Experiment()
    path = f'D:\Works\Diplom-work\Experiments\PETA,TMAG_viscosity\PETA{i}'
    file = [file for file in os.listdir(path) if '.hdf5' in file][0]
    exp.load_hdf5(*tools.input_path(f'{path}\{file}'))
    exp.log.pop()
    info, result, func = tools.regress(exp)
    exp.set_info(**info)
    del exp.info['d_D0']
    del exp.info['d_E']
    exp.save_hdf5()

In [ ]:
with pd.HDFStore('D:\Works\Diplom-work\Experiments\OCM_viscosity\OCM0\data_OCM0_up.hdf5') as file:
    log = file.get_storer('data').attrs.log 
    info= file.get_storer('data').attrs.info 

In [ ]:
info

In [ ]:
exp2 =tools.Experiment()
exp2.load_hdf5(*tools.input_path())
exp2.log

In [ ]:
exp2.info